In [1]:
#Installation of library
# !pip install -q -U tfx==1.4.0
# !pip install tensorflow==2.7.0
# !pip install tensorflow_decision_forests==0.2.1

In [2]:
# creating required directories
# !mkdir -p saved_data
# !mkdir -p model
# !mkdir -p data

In [3]:
#importing packages
import tempfile
import tensorflow as tf
import urllib.request
import os
import pandas as pd
import shutil
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_decision_forests as tfdf

from absl import logging
from pathlib import Path
from tfx import v1 as tfx
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.proto import example_gen_pb2

print(f"Tensorflow Version: {tf.__version__}")
print(f"TFX Version: {tfx.__version__}")
print(f"TFDF Version: {tfdf.__version__}")

print(f"Tensorflow Data Validation Version: {tfdv.__version__}")

logging.set_verbosity(logging.INFO)

Tensorflow Version: 2.7.0
TFX Version: 1.4.0
TFDF Version: 0.2.1
Tensorflow Data Validation Version: 1.4.0


In [4]:
DATA_DIRECTORY = 'data'
DATA_SOURCE_PATH = Path(DATA_DIRECTORY) / 'Social_Network_Ads.csv'

SAVED_DATA = 'saved_data'
DATA_TRAIN_FILENAME = Path(SAVED_DATA) / 'train.csv'
DATA_TEST_FILENAME = Path(SAVED_DATA) / 'test.csv'

PIPELINE_NAME = 'sample-pipeline'
PIPELINE_DIRECTORY = os.path.join(Path('pipelines'), PIPELINE_NAME)

METADATA_PATH = Path("metadata") / PIPELINE_NAME / "metadata.db"

SCHEMA_DIRECTORY = os.path.join(PIPELINE_DIRECTORY, 'schema')
SCHEMA_FILE_NAME = str(os.path.join(SCHEMA_DIRECTORY, 'schema.pbtxt'))

MODEL_DIRECTORY = Path('model')

# Module Paths
CONSTANTS_MODULE_PATH = 'constants.py'
TRANSFORM_MODULE_PATH = 'transform.py'
TRAINER_MODULE_PATH = 'trainer.py'

In [5]:
#Reading files
data_df = pd.read_csv(DATA_SOURCE_PATH)

# splitting the data for training and testing
data_df = data_df.sample(frac=1)
train_df = data_df[: int(len(data_df) * 0.7)]
test_df = data_df[int(len(data_df) * 0.7): ]

# removing the undesired columns from all the datasets
datasets = [train_df, test_df]
drop_columns = ['User ID']
for dataset in datasets:
    dataset.drop(drop_columns, axis=1, inplace=True)

train_df.to_csv(DATA_TRAIN_FILENAME, index=False)
test_df.to_csv(DATA_TEST_FILENAME, index=False)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
# peeking at the data
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 212 to 153
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Gender           280 non-null    object
 1   Age              280 non-null    int64 
 2   EstimatedSalary  280 non-null    int64 
 3   Purchased        280 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 10.9+ KB


In [7]:
##Common Functions
# creating the useful conversion functions

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(str(value), encoding="raw_unicode_escape")]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [8]:
# This function will be required to convert out test set 
# to compatible schema types for inferencing
def _examples(df):
    examples = []
    for index, row in df.iterrows():
        features = {
            "Gender": _bytes_feature(row['Gender']),
            "Age": _int64_feature(row['Age']),
            "EstimatedSalary": _int64_feature(row['EstimatedSalary']),
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=features))
        examples.append(example_proto.SerializeToString())

    return examples

### Running components interactively

In [9]:
from tfx.orchestration.experimental.interactive.interactive_context import (InteractiveContext)

context = InteractiveContext()

### Data Ingestion

In [10]:
output = tfx.proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     tfx.proto.SplitConfig.Split(name="train", hash_buckets=3),
                                                     tfx.proto.SplitConfig.Split(name="eval", hash_buckets=1)
    ]))
example_gen = tfx.components.CsvExampleGen(input_base=SAVED_DATA, output_config=output)

context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data saved_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:2,total_bytes:6956,xor_checksum:0,sum_checksum:3294124042"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Generating Statistics

In [11]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


### Generating Data Schema

In [12]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
I0312 05:35:24.499311  3384 rdbms_metadata_access_object.cc:686] No property is defined for the Type
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'Gender',STRING,required,,'Gender'
'Age',INT,required,,-
'EstimatedSalary',INT,required,,-
'Purchased',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"


### Updating the schema

In [13]:
schema = tfdv.load_schema_text(
    os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, "schema.pbtxt")
)

# adding the needed environments
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# removing the `Purchased` column from the serving environment
tfdv.get_feature(schema, "Purchased").not_in_environment.append("SERVING")

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Gender',STRING,required,,'Gender'
'Age',INT,required,,-
'EstimatedSalary',INT,required,,-
'Purchased',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"


In [14]:
!mkdir -p {SCHEMA_DIRECTORY}

tfdv.write_schema_text(schema, SCHEMA_FILE_NAME)

# loading the updated schema using the importer node.
schema_importer = tfx.dsl.Importer(
    source_uri=str(SCHEMA_DIRECTORY),
    artifact_type=tfx.types.standard_artifacts.Schema
).with_id("schema_importer")

context.run(schema_importer)
context.show(schema_importer.outputs['result'])

INFO:absl:Running driver for schema_importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: pipelines/sample-pipeline/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for schema_importer
INFO:absl:Running publisher for schema_importer
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'Gender',STRING,required,,'Gender'
'Age',INT,required,,-
'EstimatedSalary',INT,required,,-
'Purchased',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"


### Data Validation

In [15]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']
)

context.run(example_validator)
context.show(example_validator.outputs['anomalies'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/ExampleValidator/anomalies/5/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/ExampleValidator/anomalies/5/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


### Data Transformation

In [16]:
%%writefile {CONSTANTS_MODULE_PATH}
LABEL = 'Purchased'

Writing constants.py


In [17]:
%%writefile {TRANSFORM_MODULE_PATH}

import tensorflow as tf
import tensorflow_transform as tft

import constants

LABEL = constants.LABEL

def preprocessing_fn(inputs):

    outputs = dict()
    outputs['Age'] = inputs['Age']
    outputs['EstimatedSalary'] = inputs['EstimatedSalary']
    # converting the `Gender` into label encoded column.
    outputs['Gender'] = tf.cast(tf.equal(inputs['Gender'], 'male'), tf.int64)

    outputs[LABEL] = inputs[LABEL]

    return outputs

Writing transform.py


In [18]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=os.path.abspath(TRANSFORM_MODULE_PATH),
)
context.run(transform, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/MLOps_Pipeline_project/transform.py' (including modules: ['constants', 'transform']).
INFO:absl:User module package has hash fingerprint version e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmppzyjecgv/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp_w_lno38', '--dist-dir', '/tmp/tmplnxl1xj4']
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@/

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying transform.py -> build/lib
installing to /tmp/tmp_w_lno38
running install
running install_lib
copying build/lib/constants.py -> /tmp/tmp_w_lno38
copying build/lib/transform.py -> /tmp/tmp_w_lno38
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp_w_lno38/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3.7.egg-info
running inst

INFO:absl:Successfully installed '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform@/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmp67zoq1lh', '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85

Processing /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl


INFO:absl:Successfully installed '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl'.
INFO:absl:Installing '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpduhkaif4', '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Transform-0.0+e91a37d91407d0ae52616ab9833d7bc7403f4f6ac14dccd81c95c85c68fe17fa-py3-none-any.whl']


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 23
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 24
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 9
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Transform/pre_transform_schema/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"


In [19]:
#We can take a look at the transformed examples from our data.
train_uri = os.path.join(
    transform.outputs['transformed_examples'].get()[0].uri,
    'Split-train'
)

tfrecord_filenames = [
                      os.path.join(train_uri, name) for name in os.listdir(train_uri)
]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    print(example)

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 59
      }
    }
  }
  feature {
    key: "EstimatedSalary"
    value {
      int64_list {
        value: 42000
      }
    }
  }
  feature {
    key: "Gender"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Purchased"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 47
      }
    }
  }
  feature {
    key: "EstimatedSalary"
    value {
      int64_list {
        value: 25000
      }
    }
  }
  feature {
    key: "Gender"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Purchased"
    value {
      int64_list {
        value: 1
      }
    }
  }
}

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 26
      }
    }
  }
  feature {
    key: "EstimatedSalary"
    value {
      int64_list {
 

### Model Training

In [20]:
%%writefile {TRAINER_MODULE_PATH}

import tensorflow as tf
import tensorflow_decision_forests as tfdf
import tensorflow_transform as tft

from absl import logging
from tensorflow.keras import layers, Model, optimizers, losses, metrics
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from typing import List, Text

import constants

LABEL = constants.LABEL
BATCH_SIZE = 32
EPOCHS = 50

def _input_fn(
    file_pattern: List[Text],
    data_accessor: tfx.components.DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    batch_size: int,
) -> tf.data.Dataset:

    """
    Generates a dataset of features that can be used to train
    and evaluate the model.

    Args:
        file_pattern: List of paths or patterns of input data files.
        data_accessor: An instance of DataAccessor that we can use to
            convert the input to a RecordBatch.
        tf_transform_output: The transformation output.
        batch_size: The number of consecutive elements that we should
            combine in a single batch.

    Returns:
        A dataset that contains a tuple of (features, indices) where 
            features is a dictionary of Tensors, and indices is a single
            Tensor of label indices.
    """

    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
        schema=tf_transform_output.raw_metadata.schema,
    )

    tft_layer = tf_transform_output.transform_features_layer()

    def apply_transform(raw_features):
        transformed_features = tft_layer(raw_features)
        transformed_label = transformed_features.pop(LABEL)
        return transformed_features, transformed_label
    
    return dataset.map(apply_transform).repeat()

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """
    Returns a function that parses a serialized tf.Example and applies
    the transformations during inference.
    Args:
        model: The model that we are serving.
        tf_transform_output: The transformation output that we want to 
            include with the model.
    """
    
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        
        required_feature_spec = {
            k: v for k, v in feature_spec.items() if k != LABEL
        }

        parsed_features = tf.io.parse_example(
            serialized_tf_examples,
            required_feature_spec
        )

        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn

def _model() -> tf.keras.Model:
    inputs = [
              layers.Input(shape=(1,), name="Age"),
              layers.Input(shape=(1,), name="EstimatedSalary"),
              layers.Input(shape=(1,), name="Gender")
    ]

    x = layers.concatenate(inputs)
    x = layers.Dense(8, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=optimizers.Adam(1e-2),
        loss="binary_crossentropy",
        metrics=[metrics.BinaryAccuracy()],
    )

    model.summary(print_fn=logging.info)
    return model

def run_fn(fn_args: tfx.components.FnArgs):
    """
    The callback function that will be called by the Trainer component
    to train the model using the suplied arguments.

    Args:
        fn_args: A collection of name/value pairs representing the 
            arguments to train the model.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=BATCH_SIZE,
    )

    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=BATCH_SIZE,
    )

    model = _model()
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        epochs=EPOCHS
    )

    # We need to modify the default signature to include the transform layer in 
    # the computational graph.
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(model, tf_transform_output),
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

Writing trainer.py


In [21]:
trainer = tfx.components.Trainer(
    examples=example_gen.outputs["examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=5),
    module_file=os.path.abspath(TRAINER_MODULE_PATH),
)
context.run(trainer, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/MLOps_Pipeline_project/trainer.py' (including modules: ['constants', 'trainer', 'transform']).
INFO:absl:User module package has hash fingerprint version 00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a325945eb3a04.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpnsvwn5t7/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmphevotpor', '--dist-dir', '/tmp/tmp7e12j0dd']
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Trainer-0.0+00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a325945eb3a04-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'traine

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
installing to /tmp/tmphevotpor
running install
running install_lib
copying build/lib/constants.py -> /tmp/tmphevotpor
copying build/lib/transform.py -> /tmp/tmphevotpor
copying build/lib/trainer.py -> /tmp/tmphevotpor
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmphevotpor/tfx_user_code_Trainer-0.0+00ab6b6a4a2cdb18b4cab7250e788

INFO:absl:Successfully installed '/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/_wheels/tfx_user_code_Trainer-0.0+00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a325945eb3a04-py3-none-any.whl'.


INFO:absl:Training model.
INFO:absl:Feature Gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature EstimatedSalary has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Purchased has a shape dim {
  size: 1
}
. Setting to DenseTensor.


INFO:tensorflow:tensorflow_text is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Feature Gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature EstimatedSalary has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Purchased has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl: Layer (type)                   Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl: Age (InputLayer)               [(None, 1)]          0           []                               
INFO:absl:                                                                                                  
INFO:absl: EstimatedSalary (InputLayer)   [(None,

Epoch 1/50
100/100 [==============================] - 2s 3ms/step - loss: 799.0636 - binary_accuracy: 0.5200 - val_loss: 40.8924 - val_binary_accuracy: 0.3187
Epoch 2/50
100/100 [==============================] - 0s 2ms/step - loss: 134.7023 - binary_accuracy: 0.5047 - val_loss: 19.3253 - val_binary_accuracy: 0.7188
Epoch 3/50
100/100 [==============================] - 0s 2ms/step - loss: 90.3488 - binary_accuracy: 0.4894 - val_loss: 150.2175 - val_binary_accuracy: 0.7063
Epoch 4/50
100/100 [==============================] - 0s 2ms/step - loss: 127.6688 - binary_accuracy: 0.4928 - val_loss: 52.0924 - val_binary_accuracy: 0.6938
Epoch 5/50
100/100 [==============================] - 0s 2ms/step - loss: 89.0378 - binary_accuracy: 0.5159 - val_loss: 47.3812 - val_binary_accuracy: 0.6812
Epoch 6/50
100/100 [==============================] - 0s 2ms/step - loss: 62.8836 - binary_accuracy: 0.5097 - val_loss: 93.6148 - val_binary_accuracy: 0.7000
Epoch 7/50
100/100 [============================

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Trainer/model/7/Format-Serving/assets
INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Trainer/model/7/Format-Serving. ModelRun written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Trainer/model_run/7
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 26
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 27
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Model Evaluation

In [22]:
eval_config = tfma.EvalConfig(
    model_specs=[
                 tfma.ModelSpec(
                     signature_name="serving_default",
                     preprocessing_function_names=['tft_layer'],
                     label_key="Purchased",
                 )
    ],
    metrics_specs = [
                     tfma.MetricsSpec(
                         per_slice_thresholds={
                             "binary_accuracy": tfma.PerSliceMetricThresholds(
                                 thresholds=[
                                             tfma.PerSliceMetricThreshold(
                                                 slicing_specs=[tfma.SlicingSpec()],
                                                 threshold=tfma.MetricThreshold(
                                                     value_threshold=tfma.GenericValueThreshold(
                                                         lower_bound={"value":0.7}
                                                     ),
                                                     change_threshold=tfma.GenericChangeThreshold(
                                                         direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                                         absolute={"value": -1e-10},
                                                     ),
                                                 ),
                                             )
                                 ]
                             ),
                         }
                     )
    ],
    slicing_specs=[
                   tfma.SlicingSpec(),
                   tfma.SlicingSpec(feature_keys=["Gender"])
    ]
)

In [23]:
model_resolver = tfx.dsl.Resolver(
    strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
    model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessings=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
).with_id("latest_blessed_model_resolver")

context.run(model_resolver)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessings: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [24]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    eval_config=eval_config,
    baseline_model=model_resolver.outputs["model"],
)

context.run(evaluator, enable_cache=False)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
I0312 05:46:43.490140  3384 rdbms_metadata_access_object.cc:686] No property is defined for the Type
INFO:absl:Running executor for Evaluator
I0312 05:46:43.495919  3384 rdbms_metadata_access_object.cc:686] No property is defined for the Type
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "per_slice_thresholds": {\n        "binary_accuracy": {\n          "thresholds": [\n            {\n              "slicing_specs": [\n                {}\n              ],\n              "threshold": {\n                "change_threshold": {\n                  "absolute": -1e-10,\n                  "direction": "HIGHER_IS_BETTER"\n                },\n                "value_threshold": {\n                  "lower_bound": 0.7\n                }\n              }\n            }\n          ]\n        }\n      }\n


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa9bcbc5c50> and <keras.engine.input_layer.InputLayer object at 0x7fa91d8a2850>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa9bcbc5c50> and <keras.engine.input_layer.InputLayer object at 0x7fa91d8a2850>).
INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "per_slice_thresholds": {\n        "binary_accuracy": {\n          "thresholds": [\n            {\n              "slicing_specs": [\n                {}\n              ],\n              "threshold": {\n                "change_threshold": {\n                  "absolute": -1e-10,\n                  "direction": "HIGHER_IS_BETTER"\n                },\n                "value_threshold": {\n                  "lower_bound": 0.7\n                }\n              }\n            }\n          ]\n        }\n      }\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Purchased",\n      "


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa9bcf49250> and <keras.engine.input_layer.InputLayer object at 0x7fa91db31410>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa9bcf49250> and <keras.engine.input_layer.InputLayer object at 0x7fa91db31410>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8dddf4ad0> and <keras.engine.input_layer.InputLayer object at 0x7fa8ddde1990>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8dddf4ad0> and <keras.engine.input_layer.InputLayer object at 0x7fa8ddde1990>).
INFO:absl:Evaluation complete. Results written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Evaluator/evaluation/9.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Evaluator/blessing/9.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 16
        type_id: 30
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Evaluator/evaluation/9"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 30
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 31
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Evaluator/blessing/9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Trainer/model/7"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Pushing the Model

In [25]:
pusher = tfx.components.Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=str(MODEL_DIRECTORY)
        )
    ),
)

context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
I0312 05:47:20.783195  3384 rdbms_metadata_access_object.cc:686] No property is defined for the Type
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1647064040
INFO:absl:Model written to serving path model/1647064040.
INFO:absl:Model pushed to /tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Pusher/pushed_model/10.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 18
        type_id: 33
        uri: "/tmp/tfx-interactive-2022-03-12T05_26_49.911729-96yqlmm1/Pusher/pushed_model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "model/1647064040"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1647064040"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 33
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Inferencing

In [26]:
def get_inference_fn(model_directory):
    model_directories = (d for d in os.scandir(model_directory) if d.is_dir())
    model_path = max(model_directories, key=lambda i: int(i.name)).path

    loaded_model = tf.keras.models.load_model(model_path)

    return loaded_model.signatures["serving_default"]

In [27]:
inference_fn = get_inference_fn(MODEL_DIRECTORY)
result = inference_fn(examples=tf.constant(_examples(test_df)))
print(result["output_0"].numpy())


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8dd60f250> and <keras.engine.input_layer.InputLayer object at 0x7fa8dd618190>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8dd60f250> and <keras.engine.input_layer.InputLayer object at 0x7fa8dd618190>).


[[0.3264355 ]
 [0.97265285]
 [0.6270244 ]
 [0.44954094]
 [0.6055691 ]
 [0.43268377]
 [0.9931779 ]
 [0.9590658 ]
 [0.9381055 ]
 [0.16483685]
 [0.00963226]
 [0.17854801]
 [0.5440971 ]
 [0.12227309]
 [0.20295498]
 [0.99399805]
 [0.31204042]
 [0.779172  ]
 [0.4713493 ]
 [0.06559923]
 [0.99009776]
 [0.98698556]
 [0.06094879]
 [0.60351074]
 [0.07195705]
 [0.6666118 ]
 [0.92699564]
 [0.9809886 ]
 [0.25242084]
 [0.5700159 ]
 [0.28742206]
 [0.30117548]
 [0.11593023]
 [0.23361868]
 [0.44971702]
 [0.167728  ]
 [0.8054378 ]
 [0.18791357]
 [0.17147547]
 [0.11504939]
 [0.04695523]
 [0.99569154]
 [0.8914021 ]
 [0.99695516]
 [0.01810947]
 [0.9591758 ]
 [0.55371517]
 [0.06376794]
 [0.17280924]
 [0.05554444]
 [0.46687847]
 [0.0558475 ]
 [0.18377075]
 [0.24303532]
 [0.9293905 ]
 [0.19258127]
 [0.8423981 ]
 [0.0571115 ]
 [0.09974855]
 [0.00706607]
 [0.32770076]
 [0.13857907]
 [0.9964013 ]
 [0.33311385]
 [0.49687067]
 [0.7889198 ]
 [0.18616706]
 [0.93134916]
 [0.6897022 ]
 [0.23582542]
 [0.59192103]
 [0.89

### Pipeline

In [28]:
%%writefile {SCHEMA_FILE_NAME}

feature {
  name: "Gender"
  type: BYTES
  domain: "Gender"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Age"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "EstimatedSalary"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Purchased"
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "SERVING"
  shape {
    dim {
      size: 1
    }
  }
}
string_domain {
  name: "Gender"
  value: "Female"
  value: "Male"
}
default_environment: "TRAINING"
default_environment: "SERVING"

Overwriting pipelines/sample-pipeline/schema/schema.pbtxt


### Running Pipeline

In [29]:
import tensorflow_model_analysis as tfma

def create_pipeline(
    pipeline_name: str,
    pipeline_directory: str,
    data_directory: str,
    schema_path: str,
    model_directory: str,
    metadata_path: str,
    transform_module_path: str,
    trainer_module_path: str,
    )-> tfx.dsl.Pipeline:

    output = tfx.proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     tfx.proto.SplitConfig.Split(name="train", hash_buckets=3),
                                                     tfx.proto.SplitConfig.Split(name="eval", hash_buckets=1)
    ]))
    example_gen = tfx.components.CsvExampleGen(input_base=SAVED_DATA, output_config=output)

    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples']
    )

    schema_importer = tfx.dsl.Importer(
        source_uri=str(SCHEMA_DIRECTORY),
        artifact_type=tfx.types.standard_artifacts.Schema
    ).with_id("schema_importer")

    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_importer.outputs['result']
    )

    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_importer.outputs['result'],
        module_file=os.path.abspath(TRANSFORM_MODULE_PATH),
    )

    trainer = tfx.components.Trainer(
        examples=example_gen.outputs["examples"],
        transform_graph=transform.outputs["transform_graph"],
        train_args=tfx.proto.TrainArgs(num_steps=100),
        eval_args=tfx.proto.EvalArgs(num_steps=5),
        module_file=os.path.abspath(TRAINER_MODULE_PATH),
    )

    eval_config = tfma.EvalConfig(
        model_specs=[
                    tfma.ModelSpec(
                        signature_name="serving_default",
                        preprocessing_function_names=['tft_layer'],
                        label_key="Purchased",
                    )
        ],
        metrics_specs = [
                        tfma.MetricsSpec(
                            per_slice_thresholds={
                                "binary_accuracy": tfma.PerSliceMetricThresholds(
                                    thresholds=[
                                                tfma.PerSliceMetricThreshold(
                                                    slicing_specs=[tfma.SlicingSpec()],
                                                    threshold=tfma.MetricThreshold(
                                                        value_threshold=tfma.GenericValueThreshold(
                                                            lower_bound={"value":0.7}
                                                        ),
                                                        change_threshold=tfma.GenericChangeThreshold(
                                                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                                            absolute={"value": -1e-10},
                                                        ),
                                                    ),
                                                )
                                    ]
                                ),
                            }
                        )
        ],
        slicing_specs=[
                    tfma.SlicingSpec(),
                    tfma.SlicingSpec(feature_keys=["Gender"])
        ]
    )

    model_resolver = tfx.dsl.Resolver(
        strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
        model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
        model_blessings=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
    ).with_id("latest_blessed_model_resolver")

    evaluator = tfx.components.Evaluator(
        examples=example_gen.outputs["examples"],
        model=trainer.outputs["model"],
        eval_config=eval_config,
        baseline_model=model_resolver.outputs["model"],
    )

    pusher = tfx.components.Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=tfx.proto.PushDestination(
            filesystem=tfx.proto.PushDestination.Filesystem(
                base_directory=str(MODEL_DIRECTORY)
            )
        ),
    )

    components = [
                  example_gen,
                  statistics_gen,
                  schema_importer,
                  example_validator,
                  transform,
                  trainer,
                  model_resolver,
                  evaluator,
                  pusher
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_directory,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(
            metadata_path
            ),
            components=components,
    )

In [30]:
tfx.orchestration.LocalDagRunner().run(
    create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_directory=str(PIPELINE_DIRECTORY),
        data_directory=DATA_DIRECTORY,
        schema_path=str(SCHEMA_DIRECTORY),
        model_directory=str(MODEL_DIRECTORY),
        metadata_path=str(METADATA_PATH),
        transform_module_path=TRANSFORM_MODULE_PATH,
        trainer_module_path=TRAINER_MODULE_PATH
    )
)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/jupyter/MLOps_Pipeline_project/transform.py' (including modules: ['constants', 'trainer', 'transform']).
INFO:absl:User module package has hash fingerprint version 00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a325945eb3a04.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpu1g340lb/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp_4nliwb_', '--dist-dir', '/tmp/tmptb0pf_3z']
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at 'pipelines/sample-pipeline/_wheels/tfx_user_code_Transform-0.0+00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a3259

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
installing to /tmp/tmp_4nliwb_
running install
running install_lib
copying build/lib/constants.py -> /tmp/tmp_4nliwb_
copying build/lib/transform.py -> /tmp/tmp_4nliwb_
copying build/lib/trainer.py -> /tmp/tmp_4nliwb_
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp_4nliwb_/tfx_user_code_Transform-0.0+00ab6b6a4a2

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at 'pipelines/sample-pipeline/_wheels/tfx_user_code_Trainer-0.0+00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a325945eb3a04-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@pipelines/sample-pipeline/_wheels/tfx_user_code_Trainer-0.0+00ab6b6a4a2cdb18b4cab7250e7889247768192a89405b0f424a325945eb3a04-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "Evaluator"
  value {
    beam_executable_spec {
      python_executor_spec 

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
installing to /tmp/tmptbrce0mv
running install
running install_lib
copying build/lib/constants.py -> /tmp/tmptbrce0mv
copying build/lib/transform.py -> /tmp/tmptbrce0mv
copying build/lib/trainer.py -> /tmp/tmptbrce0mv
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmptbrce0mv/tfx_user_code_Trainer-0.0+00ab6b6a4a2cdb18b4cab7250e788

INFO:absl:Processing input csv data saved_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 1 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/sample-pipeline/CsvExampleGen/examples/1"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:2,total_bytes:6956,xor_checksum:0,sum_checksum:3294124042"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "sample-pipeline:2022-03-12T05:50:34.538287:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  valu

...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8fbec1850> and <keras.engine.input_layer.InputLayer object at 0x7fa8fbef09d0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8f80c7a10> and <keras.engine.input_layer.InputLayer object at 0x7fa8f80d5990>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8f80c7a10> and <keras.engine.input_layer.InputLayer object at 0x7fa8f80d5990>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8fbf32ed0> and <keras.engine.input_layer.InputLayer object at 0x7fa8fbf014d0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8fbf32ed0> and <keras.engine.input_layer.InputLayer object at 0x7fa8fbf014d0>).
INFO:absl:Evaluation complete. Results written to pipelines/sample-pipeline/Evaluator/evaluation/8.
INFO:absl:Checking validation results.
INFO:absl:Blessing result False written to pipelines/sample-pipeline/Evaluator/blessing/8.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 8 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'blessing': [Artifact(artifact: uri: "pipelines/sample-pipeline/Evaluator/blessing/8"
custom_properties {
  key: "name"
  value {
    string_value: "sample-pipeline:2022-03-12T05:50:34.538287:Evaluator:blessing:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "ModelBlessing"
)], 'e

### Running the Inferencing

In [31]:
inference_fn = get_inference_fn(MODEL_DIRECTORY)
result = inference_fn(examples=tf.constant(_examples(test_df)))
print(result["output_0"].numpy())


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8c2e4f250> and <keras.engine.input_layer.InputLayer object at 0x7fa8c2e92d10>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fa8c2e4f250> and <keras.engine.input_layer.InputLayer object at 0x7fa8c2e92d10>).


[[0.3264355 ]
 [0.97265285]
 [0.6270244 ]
 [0.44954094]
 [0.6055691 ]
 [0.43268377]
 [0.9931779 ]
 [0.9590658 ]
 [0.9381055 ]
 [0.16483685]
 [0.00963226]
 [0.17854801]
 [0.5440971 ]
 [0.12227309]
 [0.20295498]
 [0.99399805]
 [0.31204042]
 [0.779172  ]
 [0.4713493 ]
 [0.06559923]
 [0.99009776]
 [0.98698556]
 [0.06094879]
 [0.60351074]
 [0.07195705]
 [0.6666118 ]
 [0.92699564]
 [0.9809886 ]
 [0.25242084]
 [0.5700159 ]
 [0.28742206]
 [0.30117548]
 [0.11593023]
 [0.23361868]
 [0.44971702]
 [0.167728  ]
 [0.8054378 ]
 [0.18791357]
 [0.17147547]
 [0.11504939]
 [0.04695523]
 [0.99569154]
 [0.8914021 ]
 [0.99695516]
 [0.01810947]
 [0.9591758 ]
 [0.55371517]
 [0.06376794]
 [0.17280924]
 [0.05554444]
 [0.46687847]
 [0.0558475 ]
 [0.18377075]
 [0.24303532]
 [0.9293905 ]
 [0.19258127]
 [0.8423981 ]
 [0.0571115 ]
 [0.09974855]
 [0.00706607]
 [0.32770076]
 [0.13857907]
 [0.9964013 ]
 [0.33311385]
 [0.49687067]
 [0.7889198 ]
 [0.18616706]
 [0.93134916]
 [0.6897022 ]
 [0.23582542]
 [0.59192103]
 [0.89

In [35]:
!.. ls

/bin/bash: ..: command not found


In [32]:
!zip -r filename.zip folder

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 57.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 KB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 KB 21